In [1]:
import numpy as np
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2
import nncomp
from nncomp_molecule import constants

In [3]:
OUTDIR = constants.INPUTDIR / "kfujikawa/bms-kfold"

## 既存のfoldを読み込み、InChI_lengthを付与

In [4]:
train_df = pd.read_csv(constants.COMPETITION_DATADIR / "train_labels.csv").set_index("image_id")
train_df["InChI_length"] = train_df.InChI.apply(len)
kfold_df = pd.read_csv(OUTDIR / "5fold.csv").set_index("image_id")
kfold_df["InChI_length"] = train_df.InChI_length[kfold_df.index]
kfold_df

,fold,InChI_length
image_id,,
000011a64c74,0,81
000019cc0cd2,3,155
0000252b6d2b,0,158
000026b49b7e,4,147
000026fc6c36,0,96
...,...,...
ffffe824f539,0,96
ffffea1ebdfa,2,162
ffffee2b4534,4,159


## 新Fold番号の作成

- 旧Fold毎に、InChI_lengthで並び替え
- 2 * 旧fold + 偶数/奇数 で新Fold割り当て

In [5]:
kfold_df = kfold_df.sort_values(["fold", "InChI_length"])
kfold_df["new_fold"] = kfold_df.fold * 2 + np.arange(len(kfold_df)) % 2
kfold_df

,fold,InChI_length,new_fold
image_id,,,
ff46160b9517,0,34,0
534062494ec1,0,36,1
cbaa4844a4a4,0,36,0
9d24c1243b65,0,37,1
05fe5fc5e7b4,0,38,0
...,...,...,...
90e001a865ee,4,382,9
dc2f1bb134c0,4,382,8
afa944df8c08,4,386,9


## train_labelsと同じ順序に並び替えた上で、CSV保存

In [6]:
kfold_df = kfold_df["new_fold"][train_df.index].reset_index()
kfold_df = kfold_df.rename(columns=dict(new_fold="fold"))
kfold_df.to_csv(OUTDIR / "10fold.csv", index=False)
kfold_df

,image_id,fold
0,000011a64c74,0
1,000019cc0cd2,6
2,0000252b6d2b,0
3,000026b49b7e,9
4,000026fc6c36,0
...,...,...
2424181,ffffe824f539,0
2424182,ffffea1ebdfa,5
2424183,ffffee2b4534,8
2424184,fffff0b6eea6,6
